In [1]:
import sys
import numpy as np
import pandas as pd
import json

sys.path.append('../..')

from heat_load_calc.core import core

# モジュール core

## Outline

モジュール core は建物仕様・外気温湿度...(後で詳細に記述）...を与えることで、ステップごとのループ計算を行い、室温度・湿度、暖冷房負荷を計算するモジュールです。

## Interface

calc(input_data_dir, output_data_dir = None, show_simple_result = False, show_detail_result = False)

params:

| パラメータ名　| 型 | 概要 |
| :---: | :---: | :---: |
| input_data_dir | str | 入力データが格納されたディレクトリを指定します。 |
| output_data_dir | str | （省略可能）　　計算された室温度・湿度、暖冷房負荷等をCSVで出力するフォルダを指定します。CSVを出力しない場合は省略可能。 |
| show_simple_result | bool | (省略可能) 　計算結果（概要版）をはきだす場合はTrueにします。はきださない場合はFalseを指定しないか省略する。 |
| show_detail_result | bool | (省略可能) 　計算結果（詳細版）をはきだす場合はTrueにします。はきださない場合はFalseを指定しないか省略する。 |

returns:

計算結果の概要版と詳細版をpandas.DataFrame 形式で返します。  
2つの pandas.DataFrame 形式ファイルを、タプル形式で返します。  
(簡易版計算結果, 詳細版計算結果)

簡易版計算結果の行インデクスは日時型（例：1999-01-01 00:00:00)、列インデクスは以下の値をとります。  
行インデックスは1時間ごとのデータです。  

| 列名 | 単位 | 概要 |
| --- | --- | --- |
| temperature [degree C] | ℃ | 外気温度 |
| absolute humidity [kg/kg(DA)] | kg/kg(DA) | 外気絶対湿度 |
| normal direct solar radiation [W/m] | W/m2| 法線面直達日射量 |
| horizontal sky solar radiation [W/m2] | W/m2 | 水平面天空日射量 |
| outward radiation [W/m2] | W/m2 | 夜間放射量 |
| sun altitude [rad] | rad | 太陽高度 |
| sun azimuth [rad] | rad | 太陽方位角 |

上記、工事中

### 用意するファイル

#### mid_data_house.json

json ファイルの仕様

In [2]:
pd.read_csv('spec_mid_data.csv')

,階層1,階層2,階層3,階層4,階層5,階層6,項目名,key,入力必須条件 ○：必須項目 △：上位項目が必須の場合において必須項目,ID:value,ID:意味,型,単位,備考
0,〇,NaN,NaN,NaN,NaN,NaN,空間,spaces,〇,－,－,－,－,NaN
1,NaN,〇,NaN,NaN,NaN,NaN,室通し番号,id,〇,－,－,int,－,NaN
2,NaN,〇,NaN,NaN,NaN,NaN,室名,name,〇,－,－,str,－,NaN
3,NaN,〇,NaN,NaN,NaN,NaN,室気積,volume,〇,－,－,float,m3,NaN
4,NaN,〇,NaN,NaN,NaN,NaN,相当隙間面積,c_value,〇,－,－,float,cm2/m2,NaN
5,NaN,〇,NaN,NaN,NaN,NaN,放射暖房の対流放熱比率,beta,〇,－,－,float,－,NaN
6,NaN,〇,NaN,NaN,NaN,NaN,換気,ventilation,〇,－,－,－,－,NaN
7,NaN,NaN,〇,NaN,NaN,NaN,機械換気流入量,mechanical,〇,－,－,float,m3/sec,NaN
8,NaN,NaN,〇,NaN,NaN,NaN,隣室間換気流入量,next_spaces,〇,－,－,List[float],m3/sec,配列番号と室id番号が対応している
9,NaN,NaN,〇,NaN,NaN,NaN,自然風利用時の換気量,natural,〇,－,－,float,m3/sec,NaN


json ファイルの記述例

基本的なjsonファイルの書き方は以下のとおり。

{  
    'spaces': [  
        {'id': 0, ...その他の情報...},  
        {'id': 1, ...その他の情報...},  
        ...  
        {'id': 10, ...その他の情報...}  
    ],       
    'boundaries': [
        {'id': 0, ...その他の情報...},  
        {'id': 1, ...その他の情報...},  
        ...  
        {'id': 100, ...その他の情報...}  
    ]  
} 

In [3]:
# スペース1つ分の書き方例
with open('core_data_tutorial/mid_data_house.json') as f:
    dj = json.load(f)

dj['spaces'][0]

{'id': 0,
 'name': 'main_occupant_room',
 'volume': 70.92,
 'c_value': 2.0,
 'beta': 0.0,
 'ventilation': {'mechanical': 0.016666666666666666,
  'next_spaces': [0.0, 0.0, 0.0],
  'natural': 0.0985},
 'furniture': {'heat_capacity': 893592.0,
  'heat_cond': 196.59024,
  'moisture_capacity': 1191.4560000000001,
  'moisture_cond': 2.1446208},
 'equipment': {'heating': {'radiative': {'installed': False},
   'convective': {}},
  'cooling': {'radiative': {'installed': False},
   'convective': {'q_min': 500.0,
    'q_max': 6049.891511,
    'v_min': 11.086608037753514,
    'v_max': 20.15746915955184}}}}

In [4]:
# 境界1つ分の書き方例
with open('core_data_tutorial/mid_data_house.json') as f:
    dj = json.load(f)

dj['boundaries'][0]

{'id': 0,
 'name': 'integrated_boundary0',
 'sub_name': 'north_exterior_wall',
 'is_ground': 'false',
 'connected_space_id': 0,
 'area': 4.9775,
 'phi_a0': 0.07082379236222458,
 'phi_a1': [4.507057985882911e-05,
  -0.0009266420480683226,
  0.015710747070284452,
  0.1371664403861094,
  -0.11163144218064561,
  0.09174099947400259,
  -0.022630779908299824,
  0.0033885297113725936,
  0.0,
  0.0,
  0.0,
  0.0],
 'phi_t0': 0.001598173949854509,
 'phi_t1': [4.014649140609802e-05,
  -0.0008192243722720102,
  0.013605804259968005,
  0.1088867059986907,
  -0.19238689071678203,
  0.10296731413100914,
  -0.022435410952413436,
  0.0029186857277683836,
  0.0,
  0.0,
  0.0,
  0.0],
 'r': [0.9982016190284373,
  0.99291521625209,
  0.9723062034557024,
  0.8950058718522789,
  0.6452076990440302,
  0.17714441059303657,
  0.0010735761574841769,
  1.869406715200079e-12,
  0.0,
  0.0,
  0.0,
  0.0],
 'h_i': 9.090909090909092,
 'flr': 0.0,
 'is_solar_absorbed': 'False',
 'f_mrt_hum': 0.027134347293875108,
 '

# Example

In [5]:
ds, dd = core.calc(input_data_dir='core_data_tutorial')

助走計算（土壌のみ）
助走計算（建物全体）
本計算
ログ作成


In [6]:
ds.head()

,外気温度[℃],外気絶対湿度[kg/kg(DA)],main_occupant_room_運転状態,main_occupant_room_空気温度[℃],main_occupant_room_室絶対湿度[kg/kg(DA)],main_occupant_room_室作用温度[℃],main_occupant_room_対流空調顕熱負荷[W],main_occupant_room_放射空調顕熱負荷[W],main_occupant_room_対流空調潜熱負荷[W],other_occupant_room_運転状態,...,other_occupant_room_対流空調顕熱負荷[W],other_occupant_room_放射空調顕熱負荷[W],other_occupant_room_対流空調潜熱負荷[W],non_occupant_room_運転状態,non_occupant_room_空気温度[℃],non_occupant_room_室絶対湿度[kg/kg(DA)],non_occupant_room_室作用温度[℃],non_occupant_room_対流空調顕熱負荷[W],non_occupant_room_放射空調顕熱負荷[W],non_occupant_room_対流空調潜熱負荷[W]
1989-01-01 00:00:00,2.41,0.0,OperationMode.STOP_CLOSE,14.97,0.0040,13.97,0.0,0.0,0.0,OperationMode.HEATING,...,11067.0,0.0,0.0,OperationMode.STOP_CLOSE,5.62,0.1644,6.27,0.0,0.0,0.0
1989-01-01 01:00:00,2.71,0.0,OperationMode.STOP_CLOSE,12.75,0.0040,11.99,0.0,0.0,0.0,OperationMode.HEATING,...,11632.0,0.0,0.0,OperationMode.STOP_CLOSE,5.27,0.1673,5.89,0.0,0.0,0.0
1989-01-01 02:00:00,2.86,0.0,OperationMode.STOP_CLOSE,11.25,0.0040,10.66,0.0,0.0,0.0,OperationMode.HEATING,...,11978.0,0.0,0.0,OperationMode.STOP_CLOSE,4.95,0.1702,5.56,0.0,0.0,0.0
1989-01-01 03:00:00,2.65,0.0,OperationMode.STOP_CLOSE,10.12,0.0039,9.67,0.0,0.0,0.0,OperationMode.HEATING,...,12337.0,0.0,0.0,OperationMode.STOP_CLOSE,4.68,0.1730,5.27,0.0,0.0,0.0
1989-01-01 04:00:00,2.40,0.0,OperationMode.STOP_CLOSE,9.21,0.0039,8.86,0.0,0.0,0.0,OperationMode.HEATING,...,12708.0,0.0,0.0,OperationMode.STOP_CLOSE,4.41,0.1759,5.00,0.0,0.0,0.0


In [7]:
dd.head()

,外気温度[℃],外気絶対湿度[kg/kg(DA)],main_occupant_room_運転状態,main_occupant_room_在室状況,main_occupant_room_空気温度[℃],main_occupant_room_室相対湿度[%],main_occupant_room_室絶対湿度[kg/kg(DA)],main_occupant_room_室MRT[℃],main_occupant_room_室作用温度[℃],main_occupant_room_着衣量[clo],...,non_occupant_room_integrated_boundary8_表面放射熱流[W],non_occupant_room_integrated_boundary0_表面対流熱流[W],non_occupant_room_integrated_boundary1_表面対流熱流[W],non_occupant_room_integrated_boundary2_表面対流熱流[W],non_occupant_room_integrated_boundary3_表面対流熱流[W],non_occupant_room_integrated_boundary4_表面対流熱流[W],non_occupant_room_integrated_boundary5_表面対流熱流[W],non_occupant_room_integrated_boundary6_表面対流熱流[W],non_occupant_room_integrated_boundary7_表面対流熱流[W],non_occupant_room_integrated_boundary8_表面対流熱流[W]
1989-01-01 00:00:00,2.300,0.003200,OperationMode.STOP_CLOSE,False,15.880388,36.451059,0.004065,13.753568,14.824656,0.7,...,-360.640119,-89.188074,29.584679,-13.374172,-33.950229,23.936264,-24.574440,-92.595550,-253.578612,-317.252002
1989-01-01 00:15:00,2.375,0.003225,OperationMode.STOP_CLOSE,False,15.239571,37.874433,0.004053,13.175415,14.223264,0.7,...,-367.053618,-87.807722,28.361424,-13.154875,-33.395777,22.947004,-24.434524,-91.029231,-240.570158,-317.854118
1989-01-01 00:30:00,2.450,0.003250,OperationMode.STOP_CLOSE,False,14.655859,39.221203,0.004043,12.649779,13.675146,0.7,...,-373.188965,-86.216563,27.136536,-12.889129,-32.729608,21.956443,-24.239968,-89.643408,-229.226074,-318.491556
1989-01-01 00:45:00,2.525,0.003275,OperationMode.STOP_CLOSE,False,14.121641,40.497316,0.004032,12.175686,13.176360,0.7,...,-379.018835,-84.710149,25.910277,-12.640491,-32.104854,20.964803,-24.021969,-88.396324,-219.089478,-319.225413
1989-01-01 01:00:00,2.600,0.003300,OperationMode.STOP_CLOSE,False,13.422160,42.278733,0.004022,11.725074,12.602358,0.7,...,-384.404014,-83.056759,24.746232,-12.362244,-31.408331,20.023443,-23.708643,-86.761036,-209.477045,-319.599938
